# <font color=blue>Immigration and Employment</font>

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, linregress
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.formula.api as smf
import sqlite3
import glob
import seaborn as sns
sns.set_style("whitegrid", rc={'axes.linewidth': 2.5})
sns.set_context('notebook', font_scale=1.45, rc={"lines.linewidth": 3, "figure.figsize" : (7, 3)})

## Cleaning the data

### Data from Integrated Public Use Microdata Series (IPUMS)

We downloaded data from 2007 to 2016 in five different files (each with two years) as the files are too large. We selected the following variables: age (AGE), sex (SEX), migration status (MIGRATE1), citizenship (CITIZEN), hispanic status (HISPANIC), state (STATEFIP), county (COUNTYFIPS), and weight of population (PERWT)

To clean the five files we used a loop as the variables are the same for all files:

In [3]:
files=["2007", "2009", "2011", "2013", "2015"]
for filename in files:    
    df = pd.read_csv("IPUMS"+ filename + '.csv', 
                     usecols=["YEAR", "STATEFIP", "COUNTYFIPS", "PERWT", "SEX", "AGE", "HISPAN", "CITIZEN", "MIGRATE1"])
    #select persons that are not US citizens and that 1 year ago were living abroad
    #select only US states, do not consider Puerto Rico, Alaska, Virgin Islands, Hawaii
    #do not consider counties whose code is zero
    #do not consider persons that did not respond about their race (HISPAN=9)
    df2 = df[(df.CITIZEN == 3) & (df.MIGRATE1 == 4) & (df.HISPAN != 9) & (df.STATEFIP != 2) & (df.STATEFIP != 3)
                        & (df.STATEFIP != 7) & (df.STATEFIP != 14) & (df.STATEFIP != 15) & (df.STATEFIP != 43) 
                        & (df.STATEFIP != 52) & (df.COUNTYFIPS > 0)]
    #rename variables so it is easier to calculate proportions
    #the code for persons that are hispanic is set to zero
    df2["HISPAN2"] = 0
    #the code for persons that are not hispanic (0) is set to one
    df2.loc[df2["HISPAN"] == 0,  "HISPAN2"] = 1
    #the code for males is set to zero and the code for females (2) is set to one
    df2["SEX2"] = 0
    df2.loc[df2["SEX"] == 2,  "SEX2"] = 1
    #group by county, state, and year
    #calculate the mean of age, sex, and hispanic people (proportion), the number of migrants, and the sum of the weights (perwt)
    df3 = df2.groupby(["COUNTYFIPS", "STATEFIP", "YEAR"])
                    .agg({'AGE': 'mean', 'SEX2': 'mean', 'HISPAN2': 'mean', 'MIGRATE1': 'count', 'PERWT': 'sum'}).reset_index()
    #convert COUNTYFIPS to string
    df3.COUNTYFIPS = df3.COUNTYFIPS.astype(str)
    #convert STATEFIP to string
    df3.STATEFIP = df3.STATEFIP.astype(str)
    #add leading zeros to COUNTYFIPS so that the total number of digits is three
    df3['COUNTYFIPS'] = df3['COUNTYFIPS'].apply(lambda x: x.zfill(3))
    #add leading zeros to STATEFIP so that the total number of digits is two
    df3['STATEFIP'] = df3['STATEFIP'].apply(lambda x: x.zfill(2))
    #combine STATEFIP and COUNTYFIPS in one colummn
    df3["ID"] = df3["STATEFIP"] + df3["COUNTYFIPS"]
    #change the index to the new variable ID
    df3.set_index('ID', inplace=True)
    #save the data in separate files
    df3.to_csv('IPUMSclean' + filename + '.csv')

IndentationError: unexpected indent (<ipython-input-3-5c6dabfef5f5>, line 23)

We also used a loop to merge the five files into one single file that contains data from IPUMS (2007-2016)

In [29]:
df_allyears = pd.DataFrame()
df_allyears = pd.read_csv("IPUMSclean2007.csv", index_col=None, header=0)
df_allyears["Year_merge"] = 2007

for num in (2009,2011,2013,2015):
    filename = "IPUMSclean" + str(num) + ".csv"
    df_oneyear = pd.read_csv(filename, index_col=None, header=0)
    df_oneyear["Year_merge"] = num
    frames = [df_allyears, df_oneyear]
    df_allyears = pd.concat(frames)
df_allyears
df_allyears.to_csv('IPUMS_allyears.csv')

This file is already cleaned and ready to be merged with the dataset of the American Community Survey.

### Data from American Community Survey (ACS)

From the ACS, we downloaded the available data from 2007 to 2016 in one CSV file for each year (10 in total). From each file we selected the following variables: FIPS county number (ID), total population estimate (population), Labor force participation rate (Labor Part), employment rate (Employment), and unemployment rate (Unemployment)

In order to clean each CSV, we ran a loop through all the csv files where we selected the variables that we needed and created a clean csv file for each one

In [5]:
files=["2007", "2008", "2009", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016"]
for filename in files:    
    ACS = pd.read_csv("ACS_"+filename+ ".csv", 
                      usecols=["GEO.id2", "HC01_EST_VC01", "HC02_EST_VC01", "HC03_EST_VC01", "HC04_EST_VC01"], 
                      skiprows=[1])
    ACS = ACS.rename(columns={'GEO.id2': 'ID', 'HC01_EST_VC01': 'Population', 'HC02_EST_VC01': 'Labor Part.',
                              'HC03_EST_VC01': 'Employment', 'HC04_EST_VC01': 'Unemployment'})
    ACS.set_index('ID', inplace=True)                  
    ACS.to_csv('ACSclean' + filename + '.csv')

Once we had the clean csv files for each year we created a for loop to make one single csv file that has all the years

In [18]:
df_allyears = pd.DataFrame()
df_allyears = pd.read_csv("ACSclean2007.csv", index_col=None, header=0)
df_allyears["YEAR"] = 2007

for num in range (2008,2017):
    filename = "ACSclean" + str(num) + ".csv"
    df_oneyear = pd.read_csv(filename, index_col=None, header=0)
    df_oneyear["YEAR"] = num
    frames = [df_allyears, df_oneyear]
    df_allyears = pd.concat(frames)
df_allyears
df_allyears.to_csv('ACS_allyears.csv')

## Merging datasets from IPUMS and ACS

To merge the dataset we used the column ID as index and the code for "outer" since we are just keeping the values that are common to both datasets.

In [31]:
df_ACS = pd.read_csv("ACS_allyears.csv")
df_IPUMS = pd.read_csv("IPUMS_allyears.csv")

In [52]:
df_merged = df_ACS.merge(df_IPUMS, on = ['ID','YEAR'], how='inner')
df_merged.head()

,Unnamed: 0_x,ID,Population,Labor Part.,Employment,Unemployment,YEAR,Unnamed: 0_y,COUNTYFIPS,STATEFIP,AGE,SEX2,HISPAN2,MIGRATE1,PERWT,Year_merge
0,0,1003,135873,61.0,58.9,3.2,2007,20,3,1,22.000000,0.400000,0.900000,10,2378,2007
1,7,1073,520153,62.6,58.0,7.2,2007,348,73,1,30.666667,0.666667,1.000000,3,899,2007
2,9,1081,104436,62.6,58.9,4.8,2007,363,81,1,37.857143,0.285714,0.571429,7,479,2007
3,13,1097,310354,59.7,55.2,7.1,2007,432,97,1,31.500000,0.100000,0.400000,10,1416,2007
4,17,1117,138992,70.1,66.9,4.4,2007,497,117,1,26.666667,1.000000,1.000000,3,426,2007


We eliminate columns that we will not use.

In [53]:
df_merged.drop(df_merged.columns[[0, 7, 8, 
    9, 15]], axis=1, inplace=True)
df_merged.head()

,ID,Population,Labor Part.,Employment,Unemployment,YEAR,AGE,SEX2,HISPAN2,MIGRATE1,PERWT
0,1003,135873,61.0,58.9,3.2,2007,22.000000,0.400000,0.900000,10,2378
1,1073,520153,62.6,58.0,7.2,2007,30.666667,0.666667,1.000000,3,899
2,1081,104436,62.6,58.9,4.8,2007,37.857143,0.285714,0.571429,7,479
3,1097,310354,59.7,55.2,7.1,2007,31.500000,0.100000,0.400000,10,1416
4,1117,138992,70.1,66.9,4.4,2007,26.666667,1.000000,1.000000,3,426


In [54]:
df_merged.to_csv('ACSIPUMS_merged.csv')